In [1]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink
import seaborn as sns

import statsmodels.api as sm

from basic_tools import *

"""

jupyter nbconvert 5_association.ipynb --to script
python 5_association.py albumin 2 0

for i in {00..101};do python 5_association.py $i 2 1;done


for i in {00..10};do python 5_association.py $i 2 0;done
for i in {11..20};do python 5_association.py $i 2 0;done
for i in {21..30};do python 5_association.py $i 2 0;done
for i in {31..40};do python 5_association.py $i 2 0;done
for i in {41..50};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {61..70};do python 5_association.py $i 2 0;done
for i in {71..80};do python 5_association.py $i 2 0;done
for i in {91..90};do python 5_association.py $i 2 0;done
for i in {91..101};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done

for i in {00..101};do python 5_association.py $i 1 0,2;done
for i in {00..101};do python 5_association.py $i 1 1;done

for i in {00..20};do python 5_association.py $i 1 1;done
for i in {21..40};do python 5_association.py $i 1 1;done
for i in {41..60};do python 5_association.py $i 1 1;done
for i in {61..80};do python 5_association.py $i 1 1;done
for i in {81..101};do python 5_association.py $i 1 1;done

"""

'\n\njupyter nbconvert 5_association.ipynb --to script\npython 5_association.py albumin 2 0\n\nfor i in {00..101};do python 5_association.py $i 2 1;done\n\n\nfor i in {00..10};do python 5_association.py $i 2 0;done\nfor i in {11..20};do python 5_association.py $i 2 0;done\nfor i in {21..30};do python 5_association.py $i 2 0;done\nfor i in {31..40};do python 5_association.py $i 2 0;done\nfor i in {41..50};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {61..70};do python 5_association.py $i 2 0;done\nfor i in {71..80};do python 5_association.py $i 2 0;done\nfor i in {91..90};do python 5_association.py $i 2 0;done\nfor i in {91..101};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\n\nfor i in {00..101};do python 5_association.py $i 1 0,2;done\nfor i in {00..101};do py

In [2]:
plink_KCHIP_HLA_AA_SNP_1000G=PyPlink(plink_KCHIP_HLA_AA_SNP_1000G_path)
plink_KCHIP_HLA_AA_SNP_1000G_fam=plink_KCHIP_HLA_AA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_KCHIP_HLA_AA_SNP_1000G_bim=plink_KCHIP_HLA_AA_SNP_1000G.get_bim()

In [3]:
phenotypes=pd.read_csv(pheno_all_file_path,sep='\t')
phenotypes=phenotypes.set_index('ID').loc[plink_KCHIP_HLA_AA_SNP_1000G_fam['IID']]

binary_continuous_traits=sorted(phenotypes.columns[~phenotypes.columns.str.contains('x_ray')])    

In [4]:
#HLA_names

In [5]:
def get_a1_freq_case_control(row):
    dosage=plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(row['marker_name']).astype(float)
    dosage[dosage==-1]=np.nan
    if plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a1']==row['A1'] and plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a2']==row['A2']:
        pass
    elif plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a1']==row['A2'] and plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a2']==row['A1']:
        dosage=2-dosage
        
    dosage_control=dosage[pheno['pheno']==1]
    dosage_case=dosage[pheno['pheno']==2]
    
    a1_freq_case=(2*(dosage_case==2).sum()+1*(dosage_case==1).sum())/(2*dosage_case.shape[0])
    a1_freq_control=(2*(dosage_control==2).sum()+1*(dosage_control==1).sum())/(2*dosage_control.shape[0])
    
    return a1_freq_case,a1_freq_control

In [6]:
def name_to_pos(name):
    if name[:3]=='AA_':
        return int(name.split('_')[3])
    elif name[:4]=='HLA_':
        return plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_'+name.split('*')[0].split('_')[1]+'*')].iloc[0]['pos']
    else:
        raise

In [10]:

assoc_result_list=[]
for phenotype_name in binary_continuous_traits:
    if phenotype_name=='sex' or phenotype_name=='age':
        continue
    data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'
      

    for step_idx_sub in range(1,100):
        #print(step_idx_sub)
        if os.path.exists(data_out_assoc_phenotype_path+'step_{:02d}.cond.stop'.format(step_idx_sub)):
            print('meets end',step_idx_sub)
            break
        result_merge=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.merge.result.tsv'.format(step_idx_sub),sep='\t')
        assoc_result_list.append(result_merge.sort_values('P').iloc[0])
        """
        GAT_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.GAT.result.tsv'.format(step_idx_sub),sep='\t')
        GAT_result['POS']=GAT_result['marker_name'].map(name_to_pos)

        plink_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}'.format(step_idx_sub,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t')
        plink_result_munge=plink_result[plink_result['TEST']=='ADD'].drop(columns='#CHROM').rename(columns={'TEST':'term','ID':'marker_name','BETA':'coef','SE':'std','T_STAT':'Z','Z_STAT':'Z','OBS_CT':'nobs'})
        plink_result_munge['A2']=plink_result_munge.apply(lambda x: x['ALT'] if x['A1']==x['REF'] else x['REF'],axis=1)
        if phenotype_type=='binary':
            assert set(pheno['pheno'].unique())=={-9,1,2}
            a1_freq_case_control=plink_result_munge.apply(get_a1_freq,axis=1)
            plink_result_munge['A1_freq_case']=[i[0] for i in a1_freq_case_control]
            plink_result_munge['A1_freq_control']=[i[1] for i in a1_freq_case_control]
        else:
            plink_result_munge['A1_freq_case'],plink_result_munge['A1_freq_control']=np.nan,np.nan
        plink_result_munge=plink_result_munge.drop(columns=['REF','ALT'])
        plink_result_munge['note']='unphased bialleic'

        result_merge=pd.concat([GAT_result,plink_result_munge],sort=True)
        result_merge['phenotype_name']=phenotype_name
        if phenotype_type=='binary':
            result_merge['samples(case/control)']='{}/{}'.format((pheno['pheno']==2).sum(),(pheno['pheno']==1).sum())
        else:
            result_merge['samples(case/control)']='{}'.format((pheno['pheno']!=-9).sum())

        result_merge['step']=step_idx_sub
        with open(data_out_assoc_phenotype_path+'step_{:02d}.cond'.format(step_idx_sub)) as f:
            result_merge['condition']=','.join(f.read().split())        
        result_merge=result_merge[['phenotype_name','samples(case/control)','step','condition','marker_name','note','term','POS','A1','A2','A1_freq_case','A1_freq_control','multi_allele','nobs','Z','coef','std','chisq','df','P']]
        result_merge.sort_values('POS').to_csv(data_out_assoc_phenotype_path+'step_{:02d}.merge.result.tsv'.format(step_idx_sub),sep='\t')
        """


meets end 9
meets end 2
meets end 3
meets end 3
meets end 2
meets end 2
meets end 5
meets end 3
meets end 3
meets end 2
meets end 2
meets end 4
meets end 2
meets end 2
meets end 2
meets end 2
meets end 3
meets end 4
meets end 2
meets end 2
meets end 2
meets end 3
meets end 2
meets end 2
meets end 2
meets end 2
meets end 2
meets end 2
meets end 2
meets end 2
meets end 2
meets end 4
meets end 2
meets end 2
meets end 7
meets end 3
meets end 3
meets end 2
meets end 2
meets end 2
meets end 2
meets end 2
meets end 2
meets end 2
meets end 2
meets end 3
meets end 2
meets end 3
meets end 5
meets end 2
meets end 7
meets end 6
meets end 3
meets end 5
meets end 3
meets end 5
meets end 3
meets end 2
meets end 2
meets end 2
meets end 2
meets end 6
meets end 4
meets end 3
meets end 2
meets end 3
meets end 2
meets end 2
meets end 2
meets end 2
meets end 2
meets end 2
meets end 2
meets end 7
meets end 2
meets end 3
meets end 2
meets end 3
meets end 2
meets end 5
meets end 2
meets end 2
meets end 2
meet

In [13]:
pd.concat(assoc_result_list,axis=1).transpose().to_csv('result.tsv',sep='\t')

In [11]:
assoc_result_list.append(result_merge_sorted.iloc[0])

NameError: name 'result_merge_sorted' is not defined